# Numerical tests for paper

The goal of this notebook is to create a train of ideas and experiment to reflect the logic expressed in the numerical experiments section of the paper to indicate that:
- We can use coherence on an array of sensors to detect events (impulsive and emergent)
- SVD and QR based coherence analyses are comparable in performance
- The relationship between SVD and QR based coherence analyses depends on source configuration and noise level
- QR based coherence analyses are more computationally efficient than SVD based analyses

## Numerical Experiments

In comparing the QR and SVD based coherence analyses, we test detectability depend on the following factors:
- Noise amplitude and covariance (white, colored, spatially correlated)
- Source type (impulsive, emergent)
- Number of events
- Times of arrival, separation in frequency content, and source locations for multiple events

We first test the noise with a single impulsive source, and then a single emergent source. We then combine the two source types and test multiple events coming from different frequency contents. The intuition is that once the frequency contents are separated, other factors apart from noise level should not affect the performance of the coherence analyses. Next we test the same events occurring at different times to see if time separation affects the performance. Finally, we test multiple events of similar frequency content occurring at the same and different times.



In [ ]:
import os
import sys
import time

import deepwave
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import hilbert

# import math
from typing import Optional
import torch
from torch import Tensor
from deepwave import scalar

sys.path.append("../")
sys.path.append(os.path.join(os.path.dirname(""), os.pardir, os.pardir))
import coherence_analysis.utils as f

### Source Wavelets

`Deepwave` only has a Ricker wavelet source implementation. We implement other wavelet types here for more variety in source wavelets. We implement a minimum-phase wavelet version of the *Ormsby wavelet* for impulsive sources and a *Berlage wavelet* for emergent sources.

In [ ]:
def minimum_phase_wavelet(w, dt):
    """
    Convert arbitrary real wavelet to minimum-phase using
    log-amplitude Hilbert transform.
    """
    # n = len(w)
    W = np.fft.fft(w)

    # amplitude spectrum
    A = np.abs(W)
    # avoid log(0)
    A[A == 0] = 1e-12

    # log amplitude
    logA = np.log(A)

    # Hilbert transform → phase
    # phase = np.imag(hilbert(logA))
    phase = -np.imag(hilbert(logA))

    # minimum phase spectrum
    Wmin = np.exp(logA + 1j * phase)

    # invert to time
    wmin = np.real(np.fft.ifft(Wmin))
    return wmin


# def berlage_wavelet(t, fc, alpha, n=2, phi=0):
#     """
#     Berlage wavelet
#     t < 0 → 0
#     """
#     w = np.zeros_like(t)
#     tp = t[t >= 0]
#     w[t >= 0] = (tp**n) * np.exp(-alpha * tp) * np.cos(2*np.pi*fc*tp + phi)
#     return w


def berlage_wavelet(
    freq: float,
    length: int,
    dt: float,
    peak_time: float,
    n: int = 2,
    alpha: float = 4.0,
    phi: float = 0.0,
    dtype: Optional[torch.dtype] = None,
) -> Tensor:
    """
    Berlage wavelet
    t < 0 → 0
    """
    t = np.arange(0, length) * dt
    t = t - peak_time
    w = np.zeros_like(t)
    tp = t[t >= 0]
    w[t >= 0] = (
        (tp**n) * np.exp(-alpha * tp) * np.cos(2 * np.pi * freq * tp + phi)
    )
    w = w / np.max(np.abs(w))  # normalize
    w = np.array(w, dtype=np.float32)
    w = torch.from_numpy(w)
    return w.to(dtype)


def gabor_wavelet(t, fc, alpha):
    """
    Gabor wavelet
    fc: central frequency (Hz)
    alpha: Gaussian width parameter
    """
    return np.exp(-alpha * t**2) * np.cos(2 * np.pi * fc * t)


def ormsby_wavelet(
    freqs: list, length: int, dt: float, dtype: Optional[torch.dtype] = None
) -> Tensor:
    """
    Ormsby wavelet via analytic time-domain expression
    """
    # dt = 0.005
    t = np.arange(-length // 2, length // 2) * dt
    # t = torch.arange(float(length), dtype=dtype) * dt - length//2 * dt
    f1, f2, f3, f4 = freqs
    pi_mod = np

    def sinc(x):
        # return torch.sinc(x)
        return np.sinc(x / pi_mod.pi)

    term1 = pi_mod.pi * (f4) ** 2 * sinc(pi_mod.pi * f4 * t) ** 2
    term2 = pi_mod.pi * (f3) ** 2 * sinc(pi_mod.pi * f3 * t) ** 2
    term3 = pi_mod.pi * (f2) ** 2 * sinc(pi_mod.pi * f2 * t) ** 2
    term4 = pi_mod.pi * (f1) ** 2 * sinc(pi_mod.pi * f1 * t) ** 2

    out = (term1 - term2) / (f2 - f1) - (term3 - term4) / (f4 - f3)
    # out = (term1 - term2) - (term3 - term4)
    # out = out / np.max(np.abs(out))  # normalize
    out = torch.from_numpy(out)
    return out.to(dtype)
    # return out


def min_phase_ormsby_wavelet(
    freqs: list,
    length: int,
    dt: float,
    peak_time: float,
    dtype: Optional[torch.dtype] = None,
) -> Tensor:
    """
    Minimum-phase Ormsby wavelet
    """
    w = ormsby_wavelet(freqs, length, dt, dtype)
    # w /= torch.max(torch.abs(w))  # normalize
    w_min = minimum_phase_wavelet(w, dt)
    w_min /= np.max(np.abs(w_min))  # normalize
    w_min = np.roll(w_min, int(peak_time / dt))
    w_min = np.array(w_min, dtype=np.float32)
    w_min = torch.from_numpy(w_min)
    return w_min.to(dtype)

In [ ]:
fc = 3
alpha = 5
dt = 0.005
t = np.arange(-0.2, 5, dt)

# w = berlage_wavelet(t, fc, alpha, n=3)
w = berlage_wavelet(fc, len(t), dt, 0.2, alpha=alpha, n=10, phi=0)
w = w.numpy()
# w /= np.max(np.abs(w))  # normalize

plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(t, w)
plt.title("Berlage Wavelet")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Berlage Wavelet")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Magnitude of the Berlage wavelet spectrum")
plt.legend()

In [ ]:
# corner frequencies
corner_freqs = 5, 15, 35, 45  # Hz
dt = 0.005
nt = 2 / dt
t = np.linspace(-2, 2, int(nt))
# t = np.arange(-1, 1, dt)

# w = ormsby_wavelet(corner_freqs, nt, dt)
# w = w.numpy()
# w /= np.max(np.abs(w))  # normalize
# w = minimum_phase_wavelet(w, dt)
# w = np.roll(w, int(1/dt))

w = min_phase_ormsby_wavelet(corner_freqs, int(nt), dt, peak_time=1)

plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(t, w)
plt.title("Ormsby Wavelet")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Ormsby Wavelet")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Magnitude of the Ormsby wavelet spectrum")
plt.legend()

In [ ]:
# parameters
fc = 20  # Hz
alpha = 200
dt = 0.005
t = np.arange(-1, 1, dt)

w = gabor_wavelet(t, fc, alpha)
w = minimum_phase_wavelet(w, dt)
# w = np.roll(w, -np.argmax(np.abs(w))+ 100)

plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(t, w)
plt.title("Gabor Wavelet")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Gabor Wavelet")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Magnitude of the Gabor wavelet spectrum")
plt.legend()

Choose which device we wish to run on, specify the size of the model, and load it:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Set-up the general field parameters

Specify dimensions of model and load velocity model. Here, we are using the marmousi model because it has a complex velocity structure that will generate complicated wavefields.

In [ ]:
ny = 2301
nx = 751
dx = 0.5

marmousi_offset_left = 200
marmousi_offset_right = 0
ny = min(2301 - marmousi_offset_left - marmousi_offset_right, ny)

assert marmousi_offset_left + marmousi_offset_right + ny <= 2301, (
    "Offsets exceed the model size."
)
assert min(marmousi_offset_left, marmousi_offset_right) == 0, (
    "To stay true to offsets, one side must be unrestricted i.e set to 0."
)

v = torch.from_file("marmousi_vp.bin", size=(ny + marmousi_offset_left) * nx)
v = v[marmousi_offset_left * nx :].reshape(ny, nx).to(device)

Make a plot of the velocity model:

In [ ]:
plt.imshow(v.cpu().numpy().T, cmap="seismic", aspect="auto")
plt.colorbar()

Specify receiver locations. Here, we are using a linear array of receivers at a shallow depth to simulate a surface seismic survey. `nshots` is set to 1 to simulate a single shot gather.

In [ ]:
n_shots = 1

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

### Set up source parameters and run simulation

Specify source locations and source wavelet. Here, we are using two impulsive sources to simulate seismic events. We do this step multiple times to simulate different source configurations.

#### Configuration 1: Single impulsive source
The first configuration uses a single source. We first use a minimum-phase Ormsby wavelet as the impulsive source wavelet.

In [ ]:
n_sources_per_shot = 1
d_source = 1500  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# source_amplitudes
# source_amplitudes = (
#     deepwave.wavelets.ricker(freq, nt, dt, peak_time)
#     .repeat(n_shots, n_sources_per_shot, 1)
#     .to(device)
# )
source_amplitudes = (
    min_phase_ormsby_wavelet(corner_freqs, int(nt), dt, peak_time=peak_time)
    .repeat(n_shots, n_sources_per_shot, 1)
    .to(device)
)

### Propagate wavefield to generate synthetic receiver data

This function returns:
- wavefield_nt: The wavefield at the final time step.
- wavefield_ntm1: The wavefield at the second-to-last time step.
- psiy_ntm1: PML-related wavefield.
- psix_ntm1: PML-related wavefield.
- zetay_ntm1: PML-related wavefield.
- zetax_ntm1: PML-related wavefield.
- receiver_amplitudes: The receiver amplitudes. Empty if no receivers were specified.

In [ ]:
save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "simulated_data"
)
file_path = save_directory + "/" + "config1_2hz_1_source.pt"
file_path = save_directory + "/" + "config1_5_15_35_45_hz_1_source_ormsby.pt"
if os.path.exists(file_path):
    receiver_amplitudes = torch.load(file_path)
else:
    out = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes = out[-1]
    torch.save(receiver_amplitudes, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
coherence_data = receiver_amplitudes[0].cpu().numpy()
nr, nc = coherence_data.shape
cov_len = 100
# noise_cov = np.eye(nr) + (np.diag(np.ones(nr-1), -1) + np.diag(np.ones(nr-1), 1)) * 0.5
noise_cov = np.eye(nr)
for i in range(1, cov_len + 1):
    noise_cov += (
        np.diag(np.ones(nr - i), -i) + np.diag(np.ones(nr - i), i)
    ) * np.exp(-10 * i / cov_len)
    # noise_cov += (np.diag(np.ones(nr - i), -i) + np.diag(np.ones(nr - i), i)) * (0.5 ** i)
noise = np.random.multivariate_normal(
    mean=np.zeros(nr), cov=noise_cov, size=nc
)
noise = noise.T
print(nr, nc)
noise.shape

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data = receiver_amplitudes[0].cpu().numpy()
nr, nc = coherence_data.shape
# noise = np.random.normal(scale=1, size=(nr, nc))
# normalize noise
# noise = noise / np.linalg.norm(noise)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data)) * 1
coherence_data = coherence_data + noise
# coherence_data.shape
vmin, vmax = torch.quantile(
    receiver_amplitudes[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
event_detection, _, frequencies = f.coherence(
    coherence_data, win_len, overlap, sample_interval=dt, method="exact"
)
noise_detection, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="exact"
)
t1 = time.time()
eig_time = t1 - t0 + common_time

t0 = time.time()
event_detection_qr, _, frequencies = f.coherence(
    coherence_data, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0 + common_time

t0 = time.time()
event_detection_svd, _, frequencies = f.coherence(
    coherence_data, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0 + common_time

print("Eigenvalue time: ", eig_time)
print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = np.linspace(0, 124, num_frames)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection[1:last_freq_index],
    "-o",
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection[1:last_freq_index],
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
event_freq_inds = (frequencies >= corner_freqs[1]) & (
    frequencies <= corner_freqs[2]
)
event_detection_qr[event_freq_inds]

In [ ]:
coherence_data = receiver_amplitudes[0].cpu().numpy()
nr, nc = coherence_data.shape


def get_noise(data, cov_len):
    """Generate correlated noise based on covariance length"""

    nr, nc = data.shape
    noise_cov = np.eye(nr)
    for i in range(1, cov_len + 1):
        noise_cov += (
            np.diag(np.ones(nr - i), -i) + np.diag(np.ones(nr - i), i)
        ) * np.exp(-10 * i / cov_len)

    noise = np.random.multivariate_normal(
        mean=np.zeros(nr), cov=noise_cov, size=nc
    )
    noise = noise.T
    noise = noise / np.max(np.abs(noise))

    return noise


def noisy_data(data, signal_to_noise, cov_len=5):
    """Add correlated noise to data based on signal to noise ratio"""

    noise = get_noise(data, cov_len)
    noise = noise * np.max(np.abs(data)) / signal_to_noise
    return data + noise, noise


signal_to_noise_list = [2, 1.75, 1.5, 1.25, 1, 0.75, 0.5, 0.25]
cov_len_list = [10, 50, 100, 200]

win_len = 1
overlap = 0
samples_per_sec = 1 / dt

In [ ]:
event_freq_inds = (frequencies >= corner_freqs[1]) & (
    frequencies <= corner_freqs[2]
)
event_detection_qr[event_freq_inds]

events_list = []
event_labels = []
signal_to_n_list = []
cov_len_df_list = []
for cov_len in cov_len_list:
    for signal_to_noise in signal_to_noise_list:
        for a in range(50):
            noisy_coherence_data, noise = noisy_data(
                coherence_data, signal_to_noise, cov_len=cov_len
            )

            event_detection_qr, _, frequencies = f.coherence(
                noisy_coherence_data,
                win_len,
                overlap,
                sample_interval=dt,
                method="qr",
            )
            events_list.extend(event_detection_qr[event_freq_inds])
            event_labels.extend(["signal"] * np.sum(event_freq_inds))
            signal_to_n_list.extend(
                [signal_to_noise] * np.sum(event_freq_inds)
            )
            cov_len_df_list.extend([cov_len] * np.sum(event_freq_inds))

            noise_detection_qr, _, frequencies = f.coherence(
                noise, win_len, overlap, sample_interval=dt, method="qr"
            )

            events_list.extend(noise_detection_qr)
            event_labels.extend(["noise"] * len(noise_detection_qr))
            signal_to_n_list.extend(
                [signal_to_noise] * len(noise_detection_qr)
            )
            cov_len_df_list.extend([cov_len] * len(noise_detection_qr))
import pandas as pd

df = pd.DataFrame(
    {
        "Signal_to_Noise": signal_to_n_list,
        "Event": events_list,
        "Event_Label": event_labels,
        "Covariance_Length": cov_len_df_list,
    }
)


def noise_test(
    coherence_data: np.ndarray,
    method: str,
    win_len: int,
    overlap: float,
    sample_interval: float,
    signal_to_noise_list: list,
    cov_len_list: list,
    num_of_sims: int,
):
    events_list = []
    event_labels = []
    signal_to_n_list = []
    cov_len_df_list = []
    for cov_len in cov_len_list:
        for signal_to_noise in signal_to_noise_list:
            for a in range(num_of_sims):
                noisy_coherence_data, noise = noisy_data(
                    coherence_data, signal_to_noise, cov_len=cov_len
                )

                event_detection, _, frequencies = f.coherence(
                    noisy_coherence_data,
                    win_len,
                    overlap,
                    sample_interval=sample_interval,
                    method=method,
                )

                try:
                    event_detection[event_freq_inds]
                except NameError:
                    event_freq_inds = (frequencies >= corner_freqs[1]) & (
                        frequencies <= corner_freqs[2]
                    )
                    event_detection[event_freq_inds]

                events_list.extend(event_detection[event_freq_inds])
                event_labels.extend(["signal"] * np.sum(event_freq_inds))
                signal_to_n_list.extend(
                    [signal_to_noise] * np.sum(event_freq_inds)
                )
                cov_len_df_list.extend([cov_len] * np.sum(event_freq_inds))

                noise_detection, _, _ = f.coherence(
                    noise,
                    win_len,
                    overlap,
                    sample_interval=sample_interval,
                    method=method,
                )

                events_list.extend(noise_detection)
                event_labels.extend(["noise"] * len(noise_detection))
                signal_to_n_list.extend(
                    [signal_to_noise] * len(noise_detection)
                )
                cov_len_df_list.extend([cov_len] * len(noise_detection))

    df = pd.DataFrame(
        {
            "Signal_to_Noise": signal_to_n_list,
            "Event": events_list,
            "Event_Label": event_labels,
            "Covariance_Length": cov_len_df_list,
        }
    )

    return df


df = noise_test(
    coherence_data,
    method="qr",
    win_len=win_len,
    overlap=overlap,
    sample_interval=dt,
    signal_to_noise_list=signal_to_noise_list,
    cov_len_list=cov_len_list,
    num_of_sims=50,
)

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=df,
    x="Signal_to_Noise",
    y="Event",
    hue="Covariance_Length",
    marker="o",
    errorbar=("pi", 90),
    style="Event_Label",
    err_style="bars",
)
plt.yscale("log")

In [ ]:
g = sns.FacetGrid(
    df,
    col="Covariance_Length",
    hue="Event_Label",
    height=3.5,
    aspect=1,
    palette="colorblind",
)
g.map(
    sns.lineplot,
    "Signal_to_Noise",
    "Event",
    marker="o",
    errorbar=("pi", 95),
    err_style="bars",
)
g.add_legend()

In [ ]:
# g = sns.FacetGrid(df, col="Covariance_Length", hue='Event_Label', height=3.5, aspect=1, palette='colorblind')
g = sns.FacetGrid(
    df,
    col="Covariance_Length",
    hue="Event_Label",
    height=3.5,
    aspect=1,
    palette="pastel",
)
g.map(
    sns.violinplot,
    "Signal_to_Noise",
    "Event",
    split=True,
    fill=False,
    inner="quart",
)
g.add_legend()

In [ ]:
tick_size = 12
fsize = 14
colors = ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"]

In [ ]:
i = 19
Q, R = np.linalg.qr(norm_win_spectra[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

In [ ]:
k = 23
l = 25
m = 27
j = 10

Q, R = np.linalg.qr(norm_win_spectra[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra[k])
qr_signal3 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra[l])
qr_signal4 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra[m])
qr_signal5 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra[j])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="14 Hz")
# plt.plot(qr_signal5 / np.sum(qr_signal5), "-o", color=colors[3], label="11 Hz")
# plt.plot(qr_signal4 / np.sum(qr_signal4), "-o", color=colors[2], label="10 Hz")
# plt.plot(qr_signal3 / np.sum(qr_signal3), "-o", color=colors[1], label="9 Hz")
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="8 Hz")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

## Comparison with STA/LTA method

In [ ]:
def stalta_freq(data, len_lt, len_st):
    # Does stalta on data with len_lt as the length of the longtime and len_st
    # as the length of the short time
    import scipy.signal as ss

    if data.ndim == 1:
        longtime_avg = ss.correlate(
            np.absolute(data), np.ones(len_lt), mode="valid"
        )
        shorttime_avg = ss.correlate(
            np.absolute(data[(len_lt - len_st) :]),
            np.ones(len_st),
            mode="valid",
        )
        stalta = (shorttime_avg * len_lt) / (longtime_avg * len_st)
    elif data.ndim == 2:
        nch, nsamples = data.shape
        stalta = np.empty((nch, nsamples - len_lt + 1), dtype=np.float64)
        longtime_stencil = np.ones(int(len_lt))
        shorttime_stencil = np.ones(int(len_st))
        for a in range(nch):
            longtime_avg = ss.correlate(
                np.absolute(data[a]), longtime_stencil, mode="valid"
            )
            shorttime_avg = ss.correlate(
                np.absolute(data[a, int(len_lt - len_st) :]),
                shorttime_stencil,
                mode="valid",
            )
            stalta[a] = (shorttime_avg * len_lt) / (longtime_avg * len_st)

    return stalta


stalta_data = stalta_freq(
    coherence_data,
    500,
    75,
)

In [ ]:
xax = np.array(range(stalta_data.shape[1])) * dt
plt.plot(xax, np.mean(stalta_data, axis=0), color=colors[0], linewidth=1.5)

#### Configuration 1: Part II - Emergent source
Next, we simulate an emergent source using a Berlage wavelet.

In [ ]:
n_sources_per_shot = 1
d_source = 1500  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

freq = 3
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

source_amplitudes = (
    berlage_wavelet(freq, nt, dt, peak_time, alpha=5, n=10, phi=0)
    .repeat(n_shots, n_sources_per_shot, 1)
    .to(device)
)

In [ ]:
save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "simulated_data"
)
file_path = save_directory + "/" + "config1_10hz_1_source.pt"
file_path = save_directory + "/" + "config1_3hz_emergent_source_berlage.pt"
if os.path.exists(file_path):
    receiver_amplitudes_2 = torch.load(file_path)
else:
    out_2 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=freq,
    )
    receiver_amplitudes_2 = out_2[-1]
    torch.save(receiver_amplitudes_2, file_path)

In [ ]:
# n_sources_per_shot = 5
# d_source = 0  # 20 * 4m = 80m
# first_source = 10  # 10 * 4m = 40m
# source_depth = 500  # 2 * 4m = 8m
# # first_source = int(ny/2)
# # source_depth = int(2*nx/3)

# freq = 5
# nt = 750 * 10
# dt = 0.004
# peak_time = 10 / freq

# # source_locations
# source_locations = torch.zeros(
#     n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
# )

# source_locations[..., 1] = source_depth
# source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# # source_amplitudes
# source_amplitudes = (
#     deepwave.wavelets.ricker(freq, nt, dt, peak_time)
#     .repeat(n_shots, n_sources_per_shot, 1)
#     .to(device)
# )

# # time_btnween_sources = 0.39
# time_btnween_sources = 1
# source_amplitudes = (torch.cat((deepwave.wavelets.ricker(freq, nt, dt, peak_time).repeat(n_shots, 1, 1),
#                     deepwave.wavelets.ricker(freq, nt, dt, time_btnween_sources + peak_time).repeat(n_shots, 1, 1),
#                     deepwave.wavelets.ricker(freq, nt, dt, time_btnween_sources * 2 + peak_time).repeat(n_shots, 1, 1),
#                     deepwave.wavelets.ricker(freq, nt, dt, time_btnween_sources * 3 + peak_time).repeat(n_shots, 1, 1),
#                     deepwave.wavelets.ricker(freq, nt, dt, time_btnween_sources * 4 + peak_time).repeat(n_shots, 1, 1)), dim=1)).to(device)

In [ ]:
# plt.plot(deepwave.wavelets.ricker(freq, nt, dt, peak_time))
# plt.plot(deepwave.wavelets.ricker(freq, nt, dt, peak_time+time_btnween_sources))
# plt.xlabel("Time samples")
# plt.ylabel("Amplitude")
# plt.title("Ricker wavelet")
# plt.xlim(200, 1000)

In [ ]:
# file_path = save_directory + "/" + "config2_2hz_1_emergent_source.pt"
# file_path = save_directory + "/" + "config2_2hz_1_emergent_source_1s_spacing.pt"
# file_path = save_directory + "/" + "config2_5hz_1_emergent_source_1s_spacing.pt"
# if os.path.exists(file_path):
#     receiver_amplitudes_2 = torch.load(file_path)
# else:
#     out_2 = scalar(
#         v,
#         dx,
#         dt,
#         source_amplitudes=source_amplitudes,
#         source_locations=source_locations,
#         receiver_locations=receiver_locations,
#         accuracy=8,
#         pml_freq=freq,
#     )
#     receiver_amplitudes_2 = out_2[-1]
#     torch.save(receiver_amplitudes_2, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_2[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_2.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_2.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_2 = receiver_amplitudes_2[0].cpu().numpy()
nr, nc = coherence_data_2.shape
noise = np.random.normal(scale=1, size=(nr, nc))
# normalize noise
# noise = noise / np.linalg.norm(noise)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data_2)) * 10
coherence_data_2 = coherence_data_2 + noise
# coherence_data.shape
vmin, vmax = torch.quantile(
    receiver_amplitudes_2[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_2.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data_2[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_2, frequencies = f.normalised_windowed_spectra(
    coherence_data_2, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_2 = np.matmul(
    norm_win_spectra_2, np.conjugate(norm_win_spectra_2.transpose(0, 2, 1))
)
coherence_2 = np.absolute(welch_coherence_mat_2) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
num_frames = coherence_2.shape[0]

event_detection_2 = np.empty(num_frames)
event_detection_qr_2 = np.empty(num_frames)
event_detection_svd_2 = np.empty(num_frames)

noise_detection = np.empty(num_frames)
noise_detection_qr = np.empty(num_frames)
noise_detection_svd = np.empty(num_frames)

t0 = time.time()
welch_coherence_mat_2 = np.matmul(
    norm_win_spectra_2, np.conjugate(norm_win_spectra_2.transpose(0, 2, 1))
)
coherence2_2 = np.absolute(welch_coherence_mat_2) ** 2
for d in range(num_frames):
    # eigenvals, _ = np.linalg.eig(coherence2[d])
    eigenvals = np.linalg.eigvalsh(coherence2_2[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    event_detection_2[d] = np.max(eigenvals) / np.sum(eigenvals)

    eigenvals = np.linalg.eigvalsh(coherence_noise[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    noise_detection[d] = np.max(eigenvals) / np.sum(eigenvals)
    # eig_ratios2[d] = eigenvals[0]/np.sum(eigenvals)
t1 = time.time()
eig_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    Q, R = np.linalg.qr(norm_win_spectra_2[d])
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),0.5)
    qr_approx2 = np.diag(RRH)
    event_detection_qr_2[d] = np.max(qr_approx2) / np.sum(qr_approx2)

    Q, R = np.linalg.qr(norm_win_spectra_noise[d])
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),1)
    qr_approx2 = np.diag(RRH)
    noise_detection_qr[d] = np.max(qr_approx2) / np.sum(qr_approx2)
t1 = time.time()
qr_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    # U, S, Vh = np.linalg.svd(norm_win_spectra[d], full_matrices=False)
    S = np.linalg.svd(norm_win_spectra_2[d], compute_uv=False, hermitian=False)
    # S= np.linalg.svdvals(norm_win_spectra[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    event_detection_svd_2[d] = np.max(svd_approx2) / np.sum(svd_approx2)

    S = np.linalg.svd(
        norm_win_spectra_noise[d], compute_uv=False, hermitian=False
    )
    # S = np.linalg.svdvals(norm_win_spectra_noise[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    noise_detection_svd[d] = np.max(svd_approx2) / np.sum(svd_approx2)
t1 = time.time()
svd_time = t1 - t0 + common_time

print("Eigenvalue time: ", eig_time)
print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = np.linspace(0, 124, num_frames)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_2[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_2[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_2[1:last_freq_index],
    "-o",
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)
# plt.xlim(0, 5)
plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection[1:last_freq_index],
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

## Configuration 2: Multiple sources with different frequency contents

Here, we combine the two source types used previously to simulate multiple events with different frequency contents. We use one impulsive source with a minimum-phase Ormsby wavelet and one emergent source with a Berlage wavelet.

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 0  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(nx / d_receiver))

freq = 3
corner_freqs = [10, 20, 40, 50]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 4 / freq

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1, 0] = 500
source_locations[..., 1, 1] = 500

# source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
# source_amplitudes = (
#     deepwave.wavelets.ricker(freq, nt, dt, peak_time)
#     .repeat(n_shots, n_sources_per_shot, 1)
#     .to(device)
# )


source_amplitudes = (
    torch.cat(
        (
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 1
            ).repeat(n_shots, 1, 1),
            berlage_wavelet(
                freq, nt, dt, peak_time, alpha=5, n=10, phi=0
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = save_directory + "/" + "config3_2hz_2_sources.pt"
file_path = (
    save_directory + "/" + "config2_1_impulsive_1_emergent.pt"
)  # base case, 5,15,35,45hz
file_path = (
    save_directory + "/" + "config2_1_impulsive_1_emergent2.pt"
)  # adjust peak time +.5s, 5,15,35,45hz
file_path = (
    save_directory + "/" + "config2_1_impulsive_1_emergent3.pt"
)  # adjust peak time +1s, 10,20,40,50hz, normalized wavelets
if os.path.exists(file_path):
    receiver_amplitudes_3 = torch.load(file_path)
else:
    out_3 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_3 = out_3[-1]
    torch.save(receiver_amplitudes_3, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_3[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_3.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_3.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_3 = receiver_amplitudes_3[0].cpu().numpy()
nr, nc = coherence_data_3.shape
noise = np.random.normal(scale=1, size=(nr, nc))
# normalize noise
# noise = noise / np.linalg.norm(noise)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data_3)) * 0.1
coherence_data_3 = coherence_data_3 + noise
# coherence_data.shape
vmin, vmax = torch.quantile(
    receiver_amplitudes_3[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_3.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data_3[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_3, frequencies = f.normalised_windowed_spectra(
    coherence_data_3, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_3 = np.matmul(
    norm_win_spectra_3, np.conjugate(norm_win_spectra_3.transpose(0, 2, 1))
)
coherence_3 = np.absolute(welch_coherence_mat_3) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
num_frames = coherence_3.shape[0]

event_detection_3 = np.empty(num_frames)
event_detection_qr_3 = np.empty(num_frames)
event_detection_svd_3 = np.empty(num_frames)

noise_detection = np.empty(num_frames)
noise_detection_qr = np.empty(num_frames)
noise_detection_svd = np.empty(num_frames)

t0 = time.time()
welch_coherence_mat_3 = np.matmul(
    norm_win_spectra_3, np.conjugate(norm_win_spectra_3.transpose(0, 2, 1))
)
coherence2_3 = np.absolute(welch_coherence_mat_3) ** 2
for d in range(num_frames):
    # eigenvals, _ = np.linalg.eig(coherence2[d])
    eigenvals = np.linalg.eigvalsh(coherence2_3[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    event_detection_3[d] = np.max(eigenvals) / np.sum(eigenvals)

    eigenvals = np.linalg.eigvalsh(coherence_noise[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    noise_detection[d] = np.max(eigenvals) / np.sum(eigenvals)
    # eig_ratios2[d] = eigenvals[0]/np.sum(eigenvals)
t1 = time.time()
eig_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    R = np.linalg.qr(norm_win_spectra_3[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),0.5)
    qr_approx2 = np.diag(RRH)
    event_detection_qr_3[d] = np.max(qr_approx2) / np.sum(qr_approx2)

    R = np.linalg.qr(norm_win_spectra_noise[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),1)
    qr_approx2 = np.diag(RRH)
    noise_detection_qr[d] = np.max(qr_approx2) / np.sum(qr_approx2)
t1 = time.time()
qr_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    # U, S, Vh = np.linalg.svd(norm_win_spectra[d], full_matrices=False)
    S = np.linalg.svd(norm_win_spectra_3[d], compute_uv=False, hermitian=False)
    # S= np.linalg.svdvals(norm_win_spectra[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    event_detection_svd_3[d] = np.max(svd_approx2) / np.sum(svd_approx2)

    S = np.linalg.svd(
        norm_win_spectra_noise[d], compute_uv=False, hermitian=False
    )
    # S = np.linalg.svdvals(norm_win_spectra_noise[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    noise_detection_svd[d] = np.max(svd_approx2) / np.sum(svd_approx2)
t1 = time.time()
svd_time = t1 - t0 + common_time

print("Eigenvalue time: ", eig_time)
print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = np.linspace(0, 124, num_frames)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_3[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_3[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_3[1:last_freq_index],
    "-o",
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection[1:last_freq_index],
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 2

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_3[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
Q, R = np.linalg.qr(norm_win_spectra_3[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

## Configuration 3: Same source type, different arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 0  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(nx / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1, 0] = 500
source_locations[..., 1, 1] = 500

# source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
# source_amplitudes = (
#     deepwave.wavelets.ricker(freq, nt, dt, peak_time)
#     .repeat(n_shots, n_sources_per_shot, 1)
#     .to(device)
# )


source_amplitudes = (
    torch.cat(
        (
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 2
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = save_directory + "/" + "config3_2_impulsive_sources.pt"
if os.path.exists(file_path):
    receiver_amplitudes_3 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()
nr, nc = coherence_data_4.shape
noise = np.random.normal(scale=1, size=(nr, nc))
# normalize noise
# noise = noise / np.linalg.norm(noise)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data_4)) * 3
coherence_data_4 = coherence_data_4 + noise
# coherence_data.shape
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_4, frequencies = f.normalised_windowed_spectra(
    coherence_data_4, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence_4 = np.absolute(welch_coherence_mat_4) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
num_frames = coherence_4.shape[0]

event_detection_4 = np.empty(num_frames)
event_detection_qr_4 = np.empty(num_frames)
event_detection_svd_4 = np.empty(num_frames)

noise_detection = np.empty(num_frames)
noise_detection_qr = np.empty(num_frames)
noise_detection_svd = np.empty(num_frames)

t0 = time.time()
welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence2_4 = np.absolute(welch_coherence_mat_4) ** 2
for d in range(num_frames):
    # eigenvals, _ = np.linalg.eig(coherence2[d])
    eigenvals = np.linalg.eigvalsh(coherence2_4[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    event_detection_4[d] = np.max(eigenvals) / np.sum(eigenvals)

    eigenvals = np.linalg.eigvalsh(coherence_noise[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    noise_detection[d] = np.max(eigenvals) / np.sum(eigenvals)
    # eig_ratios2[d] = eigenvals[0]/np.sum(eigenvals)
t1 = time.time()
eig_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    R = np.linalg.qr(norm_win_spectra_4[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),0.5)
    qr_approx2 = np.diag(RRH)
    event_detection_qr_4[d] = np.max(qr_approx2) / np.sum(qr_approx2)

    R = np.linalg.qr(norm_win_spectra_noise[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),1)
    qr_approx2 = np.diag(RRH)
    noise_detection_qr[d] = np.max(qr_approx2) / np.sum(qr_approx2)
t1 = time.time()
qr_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    # U, S, Vh = np.linalg.svd(norm_win_spectra[d], full_matrices=False)
    S = np.linalg.svd(norm_win_spectra_4[d], compute_uv=False, hermitian=False)
    # S= np.linalg.svdvals(norm_win_spectra[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    event_detection_svd_4[d] = np.max(svd_approx2) / np.sum(svd_approx2)

    S = np.linalg.svd(
        norm_win_spectra_noise[d], compute_uv=False, hermitian=False
    )
    # S = np.linalg.svdvals(norm_win_spectra_noise[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    noise_detection_svd[d] = np.max(svd_approx2) / np.sum(svd_approx2)
t1 = time.time()
svd_time = t1 - t0 + common_time

print("Eigenvalue time: ", eig_time)
print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = np.linspace(0, 124, num_frames)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_4[1:last_freq_index],
    "-o",
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection[1:last_freq_index],
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 2

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

## Configuration 4: Different sources, same arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 1500  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(nx / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1, 0] = 500
source_locations[..., 1, 1] = 500

# source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
# source_amplitudes = (
#     deepwave.wavelets.ricker(freq, nt, dt, peak_time)
#     .repeat(n_shots, n_sources_per_shot, 1)
#     .to(device)
# )


source_amplitudes = (
    torch.cat(
        (
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = save_directory + "/" + "config4_2_impulsive_sources_diff_loc.pt"
if os.path.exists(file_path):
    receiver_amplitudes_3 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()
nr, nc = coherence_data_4.shape
noise = np.random.normal(scale=1, size=(nr, nc))
# normalize noise
# noise = noise / np.linalg.norm(noise)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data_4)) * 3
coherence_data_4 = coherence_data_4 + noise
# coherence_data.shape
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_4, frequencies = f.normalised_windowed_spectra(
    coherence_data_4, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence_4 = np.absolute(welch_coherence_mat_4) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
num_frames = coherence_4.shape[0]

event_detection_4 = np.empty(num_frames)
event_detection_qr_4 = np.empty(num_frames)
event_detection_svd_4 = np.empty(num_frames)

noise_detection = np.empty(num_frames)
noise_detection_qr = np.empty(num_frames)
noise_detection_svd = np.empty(num_frames)

t0 = time.time()
welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence2_4 = np.absolute(welch_coherence_mat_4) ** 2
for d in range(num_frames):
    # eigenvals, _ = np.linalg.eig(coherence2[d])
    eigenvals = np.linalg.eigvalsh(coherence2_4[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    event_detection_4[d] = np.max(eigenvals) / np.sum(eigenvals)

    eigenvals = np.linalg.eigvalsh(coherence_noise[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    noise_detection[d] = np.max(eigenvals) / np.sum(eigenvals)
    # eig_ratios2[d] = eigenvals[0]/np.sum(eigenvals)
t1 = time.time()
eig_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    R = np.linalg.qr(norm_win_spectra_4[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),0.5)
    qr_approx2 = np.diag(RRH)
    event_detection_qr_4[d] = np.max(qr_approx2) / np.sum(qr_approx2)

    R = np.linalg.qr(norm_win_spectra_noise[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),1)
    qr_approx2 = np.diag(RRH)
    noise_detection_qr[d] = np.max(qr_approx2) / np.sum(qr_approx2)
t1 = time.time()
qr_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    # U, S, Vh = np.linalg.svd(norm_win_spectra[d], full_matrices=False)
    S = np.linalg.svd(norm_win_spectra_4[d], compute_uv=False, hermitian=False)
    # S= np.linalg.svdvals(norm_win_spectra[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    event_detection_svd_4[d] = np.max(svd_approx2) / np.sum(svd_approx2)

    S = np.linalg.svd(
        norm_win_spectra_noise[d], compute_uv=False, hermitian=False
    )
    # S = np.linalg.svdvals(norm_win_spectra_noise[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    noise_detection_svd[d] = np.max(svd_approx2) / np.sum(svd_approx2)
t1 = time.time()
svd_time = t1 - t0 + common_time

print("Eigenvalue time: ", eig_time)
print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = np.linspace(0, 124, num_frames)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_4[1:last_freq_index],
    "-o",
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection[1:last_freq_index],
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 2

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

## Configuration 5: Different sources, different arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 1500  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(nx / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1, 0] = 500
source_locations[..., 1, 1] = 500

# source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
# source_amplitudes = (
#     deepwave.wavelets.ricker(freq, nt, dt, peak_time)
#     .repeat(n_shots, n_sources_per_shot, 1)
#     .to(device)
# )


source_amplitudes = (
    torch.cat(
        (
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 2
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = (
    save_directory + "/" + "config4_2_impulsive_sources_diff_loc_diff_time.pt"
)
if os.path.exists(file_path):
    receiver_amplitudes_3 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()
nr, nc = coherence_data_4.shape
noise = np.random.normal(scale=1, size=(nr, nc))
# normalize noise
# noise = noise / np.linalg.norm(noise)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data_4)) * 3
coherence_data_4 = coherence_data_4 + noise
# coherence_data.shape
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_4, frequencies = f.normalised_windowed_spectra(
    coherence_data_4, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence_4 = np.absolute(welch_coherence_mat_4) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
num_frames = coherence_4.shape[0]

event_detection_4 = np.empty(num_frames)
event_detection_qr_4 = np.empty(num_frames)
event_detection_svd_4 = np.empty(num_frames)

noise_detection = np.empty(num_frames)
noise_detection_qr = np.empty(num_frames)
noise_detection_svd = np.empty(num_frames)

t0 = time.time()
welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence2_4 = np.absolute(welch_coherence_mat_4) ** 2
for d in range(num_frames):
    # eigenvals, _ = np.linalg.eig(coherence2[d])
    eigenvals = np.linalg.eigvalsh(coherence2_4[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    event_detection_4[d] = np.max(eigenvals) / np.sum(eigenvals)

    eigenvals = np.linalg.eigvalsh(coherence_noise[d])
    # eigenvals = sl.eigvalsh(coherence2[d])
    eigenvals = np.sort(eigenvals)[::-1]
    noise_detection[d] = np.max(eigenvals) / np.sum(eigenvals)
    # eig_ratios2[d] = eigenvals[0]/np.sum(eigenvals)
t1 = time.time()
eig_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    R = np.linalg.qr(norm_win_spectra_4[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),0.5)
    qr_approx2 = np.diag(RRH)
    event_detection_qr_4[d] = np.max(qr_approx2) / np.sum(qr_approx2)

    R = np.linalg.qr(norm_win_spectra_noise[d], mode="r")
    # qr_approx2 = np.sort(np.diag(np.absolute(R@(R.getH()))))[::-1]

    RRH = R @ (np.matrix(R).H)
    # diag is of sqrt(RR^*)
    # qr_approx2 = np.power(np.diag(RRH),1)
    qr_approx2 = np.diag(RRH)
    noise_detection_qr[d] = np.max(qr_approx2) / np.sum(qr_approx2)
t1 = time.time()
qr_time = t1 - t0 + common_time

t0 = time.time()
for d in range(num_frames):
    # U, S, Vh = np.linalg.svd(norm_win_spectra[d], full_matrices=False)
    S = np.linalg.svd(norm_win_spectra_4[d], compute_uv=False, hermitian=False)
    # S= np.linalg.svdvals(norm_win_spectra[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    event_detection_svd_4[d] = np.max(svd_approx2) / np.sum(svd_approx2)

    S = np.linalg.svd(
        norm_win_spectra_noise[d], compute_uv=False, hermitian=False
    )
    # S = np.linalg.svdvals(norm_win_spectra_noise[d])
    svd_approx2 = S**2
    # svd_approx2 = np.sort(S)[::-1]**2
    noise_detection_svd[d] = np.max(svd_approx2) / np.sum(svd_approx2)
t1 = time.time()
svd_time = t1 - t0 + common_time

print("Eigenvalue time: ", eig_time)
print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = np.linspace(0, 124, num_frames)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_4[1:last_freq_index],
    "-o",
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection[1:last_freq_index],
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 2

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)